# 查詢蘋果日報關鍵字搜尋，儲存至MongoDB

In [1]:
#coding:utf-8
#65001
import urllib.request
import json
import codecs
import sys
import argparse as ap
import time
import datetime
import requests
import random
import pymongo as mg
from bs4 import BeautifulSoup as bs
from urllib.parse import quote

#python main.py 八仙塵爆 2015-06-27 2015-08-24 1
#def argParse():
#    parser=ap.ArgumentParser(description='Apple Daily News Crawler')
#    parser.add_argument("keyword", help="Serch Keyword")
#    parser.add_argument("start_date", help="Start (2017-01-01)")
#    parser.add_argument("end_date", help="End (2017-02-02)" )
#    parser.add_argument("pages", help="Pages")
#    return parser.parse_args()

#args=argParse()
#keyword = args.keyword
#start_date = args.start_date
#end_date = args.end_date
#pages = args.pages

keyword = '川普'
start_date = '2018-07-01'
end_date = '2018-07-19'
pages = '1'

client = mg.MongoClient('127.0.0.1:27017')
db = client['applenews']

rs = requests.session()

def start_requests(uri):
    if( len(start_date.split("-"))==3 and len(end_date.split("-"))==3) :
        for i in range(1,int(pages)+1):
            str_page = ''+('%s' % i)
            str_rand = str(random.randint(1,99999))
            print (uri+", page="+str_page)
            parseAppleDailyNews(uri+"&page="+str_page)
            time.sleep(0.5)
      
    else:
        print ("Data format error.")


def request_uri(uri,str_page):
    #header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0','Content-Type': 'application/x-www-form-urlencoded', 'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Encoding':'gzip, deflate','Referer':'http://search.appledaily.com.tw/appledaily/search'}
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}
    data = {"searchMode":"Adv","searchType":"text","querystrA":keyword,"select":"AND","source":"","sdate":start_date,"edate":end_date,"sorttype":"1","page":str_page}
    res = rs.post(uri, data=data, headers=header)
    html_data =  res.text
    return html_data

                        
def request_uri_content(uri):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0'}
    res = rs.get(uri, headers=header)
    html_data =  res.text
    return html_data


def parseAppleDailyNews(uri):
    # https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart=2018/07/01&dateEnd=2018/07/19&querystrS=川普&page=2
    html_data =  request_uri_content(uri)
    soup = bs(html_data,'html.parser')
    
    postdate = []
    link = []
    title = []
    body = []
    for div_soup in soup.findAll('div',attrs={"class":"content"}):
        #items.append({"uri":uri+'&page='+str_page,"div_soup":str(div_soup),"updatetime":datetime.datetime.now().strftime('%Y-%m-%d')})
        for a_soup in div_soup.findAll('h2'):
            tmp_title = a_soup.text
            title.append(tmp_title.strip())
        for p_soup in div_soup.findAll('p'):
            body.append(p_soup.text)
        for li_soup in div_soup.findAll('li'):
            for a_soup in li_soup.findAll('a'):
                content_uri = a_soup.get('href').strip()
                link.append(content_uri)
                print(content_uri)
                html_data2 = request_uri_content(content_uri)
                soup2 = bs(html_data2,'html.parser')
                body_tmp = ''
                if len(soup2.findAll('div',attrs={"class":"ndArticle_margin"}))==0:
                    body.append('N/A')
                else:
                    for p_soup2 in soup2.findAll('div',attrs={"class":"ndArticle_margin"}):
                        body_tmp = p_soup2.getText().split('googletag')[0].replace('有話要說 投稿「即時論壇」','').split('.ndgKeyword')[0].split('本新聞文字')[0].strip()
                        if 'confirmOMOAdvFlag' in body_tmp:
                            body_tmp = body_tmp.split('if')[0]
                        print(body_tmp)
                        body.append(body_tmp)
#                 time.sleep(0.5)

    for time_soup in soup.findAll('time'):
        tmp_d = ''
        if len(time_soup.getText())==8:
            tmp_d = time_soup.getText()[0:4]+'-'+time_soup.getText()[4:6]+'-'+time_soup.getText()[6:8]
        #elif len(time_soup.getText())==5:
            #tmp_d = datetime.datetime.now().strftime('%Y-%m-%d')
        if len(tmp_d)>1:
            postdate.append(tmp_d)
    
    current = 0
    while current < len(postdate):
        result = db.urllib.insert_one({
                "title": title[current],
                "link":link[current],
                "body":body[current],
                "postdate":postdate[current],
                "datetime":datetime.datetime.now(),
                "updatetime":datetime.datetime.now().strftime('%Y-%m-%d')
            })
        current+=1




if __name__ == '__main__':
    uri = 'https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart='+start_date.replace('-','%2F')+'&dateEnd='+end_date.replace('-','%2F')+'&querystrS='+quote(keyword)
    items = []
    start_requests(uri);
    row_json = json.dumps(items, ensure_ascii=False)
    file = codecs.open(urllib.parse.unquote(keyword)+'.json', 'w', encoding='utf-8')
    file.write(row_json)
    file.close()
    r = requests.get(url=uri, headers={'Connection':'close'})

    print("Done")


https://tw.appledaily.com/search/result?sort=time&searchType=all&dateStart=2018%2F07%2F01&dateEnd=2018%2F07%2F19&querystrS=%E5%B7%9D%E6%99%AE, page=1
https://tw.news.appledaily.com/international/realtime/20180719/1394282/
（新增動新聞）美國司法部本周起訴一名29歲俄羅斯女子布提娜（Mariia Butina），指控她是俄國特工、企圖干涉美國大選。最新曝光的起訴書內容指出，該女子曾利用性關係來滲透重要機構和取得情報。俄國官員甚至形容她的鋒頭強過先前紅遍全球的俄國艷諜查普曼。 用學生簽證前來美國的布提娜周三在庭上被檢方指控和俄國情報單位密切接觸，美國司法部蒐證指出她刻意和一名美國共和黨黨工發展情侶關係，並以身體交換在特殊利益機構的工作職位。起訴書中並未透漏涉案黨工和特殊利益機構的身分細節，但強調布提娜透過這些滲透工作獲得情資，再將這些情報提供給一名資深俄國官員。 檢方同時公布一張布提娜和俄國情報官員共餐照片，及在她住處發現的字條，上面指出布提娜可望獲得俄國情報機構FSB的工作職位。檢方表示，聯邦調查局（FBI）幹員蒐證時發現，負責和布提娜接洽的俄國官員甚至將她和2010年在美國落網的俄國艷諜查普曼（Anna Chapman）相比。 該名官員透過簡訊向布提娜說：「你的粉絲有向你要簽名嗎？你的鋒頭完全蓋過安娜查普曼。她拿玩具槍拍照，而你是拿真槍拍照。」布提娜還在美國大選投票日一個月前透過推特（Twitter）向該名俄國官員表示，她知道「一切都得很低調小心」。在川普就職日當天，布提娜在華府自拍並將照片傳給對方，該俄國官員回說：「你這個女生超大膽！我能說什麼呢！」布提娜回傳：「我有好老師啊！」 法官表示，由於布提娜被逮捕時正準備打包離開美國，認為她有逃亡之虞而判她在受審前必須遭羈押。（桂家齊／綜合外電報導）發稿時間：08:07更新時間：21:22 跟上國際脈動，快來蘋果國際按讚  想知道更多，一定要看……​雙普閉門會議談什麼？美議員要求口譯作證挺普丁被控賣國賊　川普：我有讓他不開心竟為普丁擋箭　川普提前下台賠率急升左為布提娜（翻攝臉書），右為查普曼（資料照片）